## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

## Prepare data for model training

In [2]:
with open("../input/tps-may-data-preprocess-v1-1/TPS_May_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

del processed_data
gc.collect()

0

In [3]:
cat_cols = train_df.iloc[:,0:50].columns
train_df[cat_cols] = train_df[cat_cols].astype(int)
test_df[cat_cols] = test_df[cat_cols].astype(int)
cat_cols_indices = [train_df.columns.get_loc(col) for col in cat_cols]
print(cat_cols_indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


In [4]:
Xtrain = train_df.loc[:, train_df.columns != 'target'].copy()
Ytrain = train_df['target'].copy()
Ytrain_oh = pd.get_dummies(train_df['target']).copy()
Xtest = test_df.copy()

print("Xtrain: {} \nYtrain: {} \nYtrain_oh: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, 
                                                                   Ytrain_oh.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (99918, 951) 
Ytrain: (99918,) 
Ytrain_oh: (99918, 4) 
Xtest: (50000, 951)


20

## Build and validate the model

In [5]:
FOLD = 10
NUM_SEED = 3

# Prediction Clipping Thresholds
p_min = 0.025
p_max = 1 - p_min

np.random.seed(2021)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_cb = np.zeros((Ytrain.shape[0], 4))
y_pred_final_cb = np.zeros((Xtest.shape[0], 4))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain.values, Ytrain.values)):
        counter += 1

        train_x, train_y, train_y_oh = Xtrain.iloc[train], Ytrain.iloc[train], Ytrain_oh.iloc[train]
        val_x, val_y, val_y_oh = Xtrain.iloc[val], Ytrain.iloc[val], Ytrain_oh.iloc[val]

        model = CatBoostClassifier(
            objective='MultiClass',
            eval_metric='MultiClass',
            num_boost_round=5000,
            max_ctr_complexity=15,
            od_wait=1000, 
            od_type='Iter',
            use_best_model=True,
            bootstrap_type='Poisson',
            learning_rate=0.01465,
            reg_lambda=0.01864,
            subsample=0.68135,
            max_depth=6, 
            min_data_in_leaf=1,
            task_type='GPU',
            verbose=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  cat_features=cat_cols_indices,
                  early_stopping_rounds=200, verbose=200)

        y_pred = model.predict_proba(val_x)
        y_pred = np.clip(y_pred, p_min, p_max)
        y_pred_meta_cb[val] += y_pred
        y_pred_final_cb += model.predict_proba(Xtest)
        
        score = log_loss(val_y_oh, y_pred)
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_cb = y_pred_meta_cb / float(NUM_SEED)
y_pred_final_cb = y_pred_final_cb / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

0:	learn: 1.3775370	test: 1.3775517	best: 1.3775517 (0)	total: 40.8ms	remaining: 3m 23s
200:	learn: 1.1008356	test: 1.1072282	best: 1.1072282 (200)	total: 4.22s	remaining: 1m 40s
400:	learn: 1.0891136	test: 1.1007988	best: 1.1007988 (400)	total: 7.31s	remaining: 1m 23s
600:	learn: 1.0814564	test: 1.0979312	best: 1.0979312 (600)	total: 10.4s	remaining: 1m 15s
800:	learn: 1.0752398	test: 1.0962490	best: 1.0962490 (800)	total: 13.8s	remaining: 1m 12s
1000:	learn: 1.0696275	test: 1.0949976	best: 1.0949976 (1000)	total: 16.7s	remaining: 1m 6s
1200:	learn: 1.0646006	test: 1.0940450	best: 1.0940450 (1200)	total: 19.7s	remaining: 1m 2s
1400:	learn: 1.0602031	test: 1.0936847	best: 1.0936847 (1400)	total: 23s	remaining: 59.1s
1600:	learn: 1.0558648	test: 1.0933881	best: 1.0933855 (1598)	total: 26s	remaining: 55.1s
1800:	learn: 1.0519154	test: 1.0932078	best: 1.0932027 (1794)	total: 28.9s	remaining: 51.3s
2000:	learn: 1.0480504	test: 1.0929453	best: 1.0929423 (1998)	total: 32s	remaining: 47.9s
22

In [6]:
np.savez_compressed('./CB_Meta_Features.npz',
                    y_pred_meta_cb=y_pred_meta_cb, 
                    oof_score=oof_score,
                    y_pred_final_cb=y_pred_final_cb)

## Create submission file

In [7]:
y_pred_final_cb = np.clip(y_pred_final_cb, p_min, p_max)

test_df = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
submit_df = pd.DataFrame()
submit_df['id'] = test_df['id']
submit_df['Class_1'] = y_pred_final_cb[:,0]
submit_df['Class_2'] = y_pred_final_cb[:,1]
submit_df['Class_3'] = y_pred_final_cb[:,2]
submit_df['Class_4'] = y_pred_final_cb[:,3]
submit_df.head()

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.088001,0.619423,0.168227,0.124349
1,100001,0.084093,0.688076,0.143275,0.084555
2,100002,0.084495,0.645858,0.170326,0.099321
3,100003,0.090398,0.538620,0.270933,0.100049
4,100004,0.073982,0.624854,0.181196,0.119968


In [8]:
submit_df.to_csv("./CB_submission.csv", index=False)